In [1]:
import numpy as np
import pandas as pd
import os

# Dataset:

Netflix Price Dataset: https://www.kaggle.com/datasets/netflix-inc/netflix-prize-data

# Load Dataset

using with open() to open csv file as:
1. Deal with letters as À, É, ..., etc.
2. Some movies habe multiple titles seperarted by commas.

In [2]:
movies_df = pd.DataFrame(columns=['id', 'year', 'title'])

with open('archive/movie_titles.csv', 'rb') as file:
    for l in file.readlines():
        line_str = l.decode('utf-8', errors="ignore")
        columns = line_str.split(',')
        id_val, date_val, title_val = columns[:3]
        title_val += ', '.join(columns[3:]).strip()
        movies_df = pd.concat([movies_df, pd.DataFrame([{'id':id_val,'year': date_val,'title':title_val}])], ignore_index=True)
        
movies_df.head(3)

movies_df.to_excel("movies_data_final.xlsx")

## Merge all user rating into a single CSV

In [4]:
path = "/Users/himanshuchuri/Desktop/DataScience/Netflix/archive/"

ratings = pd.DataFrame(columns=['movie_id','customer_id','rating','date'])

movie_id = 0
for file in ['combined_data_1.txt','combined_data_2.txt','combined_data_3.txt','combined_data_4.txt','combined_data_4.txt']:
    with open(path+file) as of:
        for f in of.readlines():
            if f.endswith(':\n'):
                movie_id = f[:-2]
            else:
                cust_id, rating, date = f.split(',')
                ratings = pd.concat([ratings, pd.DataFrame([{'movie_id':movie_id,'customer_id':cust_id,'rating':rating,'date':date[:-1]}])], ignore_index=True)


KeyboardInterrupt: 

In [5]:
#ratings.to_excel("ratings_data_final.xlsx")
len(ratings)

1893847